en esta version tomo para el marketshare FEB19, si en algun mes de 2019 no esta le calcula el promedio de 2019 al final +  la arquitectura de German. si la prediccion es negativa pone 0

In [19]:
import pandas as pd
import numpy as np
import os
import random
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Bidirectional, Dense, TimeDistributed, Reshape
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from keras.layers import BatchNormalization
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
import joblib
import matplotlib.pyplot as plt
import joblib
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Bidirectional, Conv1D, MaxPooling1D, Flatten, Reshape
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='keras')  # Ignorar las advertencias de Keras



file_path= "C:/Users/Usuario/desktop/vero2/final_dataset_descr.csv"
df = pd.read_csv(file_path, sep='\t')


#Paso1: reemplazar 082019 por promedio 07 y 09
df['periodo'] = df['periodo'].astype(str).str.strip()

# Filtrar los datos por los periodos 201907, 201908 y 201909
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]

# # Pivotear los datos para tener columnas separadas para cada periodo
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()

# # Asegurar que las columnas 201907 y 201909 existen en el DataFrame
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])

# # Calcular el promedio de julio y septiembre
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)

# # Convertir de nuevo al formato largo
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], 
                                   var_name='periodo', value_name='tn')

# # Unir con el dataframe original
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)


# Paso 3: Aplicar LabelEncoder a las columnas categoricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


# Paso 4: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y descripcion

# Supongamos que 'periodo' es una columna de tipo string con formato 'YYYYMM'
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')

# Filtrar datos para diciembre de 2019
df_diciembre_2019 = df[(df['periodo'].dt.year == 2019) & (df['periodo'].dt.month == 2)]

# Agrupar las ventas del año 2019 por periodo, cat1, cat2, cat3, brand, descripcion y product_id
grouped_sales_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand','product_id'])['tn'].sum().reset_index()
#'periodo', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion', 

# Calcular el promedio de ventas por periodo para el año 2019 por grupo de cat1, cat2, cat3, brand, descripcion y product_id
#average_sales_2019 = grouped_sales_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'product_id'])['tn'].mean().reset_index()

# Calcular el promedio total de ventas por grupo de cat1, cat2, cat3, brand y descripcion para el año 2019
group_totals_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand'])['tn'].sum().reset_index()

# Unir para calcular el ratio
ratios_2019 = pd.merge(grouped_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']

# Crear un diccionario de ratios
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'product_id'])['ratio'].to_dict()



# Filtrar el DataFrame para los meses y IDs específicos
df_subset = ratios_2019[(ratios_2019['product_id'] == 20001)]

# Imprimir el resultado
print(df_subset)
df_subset.tail(10)


# Paso 5: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y customer_id
grouped_df = df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')


# Cargar los scalers guardados
scalers = joblib.load('scalers.pkl')

# Asumiendo que scaled_df ya está cargado y escalado
scaled_df['periodo'] = pd.to_datetime(scaled_df['periodo'], format='%Y%m')

# Función para crear secuencias de tiempo
def crear_secuencias(data, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(data) - n_steps - step_ahead):
        X.append(data[['tn']].iloc[i:i+n_steps].values)
        y.append(data['tn'].iloc[i+n_steps+step_ahead-1])
    return np.array(X), np.array(y)

# Función para construir el modelo LSTM bidireccional
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(input_shape)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((input_shape[0] // 2, 64)))  # Reformatear para que sea compatible con Bidirectional LSTM
    model.add(Bidirectional(LSTM(50, activation='relu')))
    model.add(Dense(1))  # Ajustado a 1 unidad de salida
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Procesar los datos en lotes para múltiples variables
def process_data_in_batches(scaled_df, n_steps, step_ahead=1):
    models = {}
    predictions = []

    # Agrupar por 'cat1', 'cat2', 'cat3', 'brand'
    for (cat1, cat2, cat3, brand), group_data in scaled_df.groupby(['cat1', 'cat2', 'cat3', 'brand']):
        group_data = group_data.sort_values(by='periodo')
        
        # Crear secuencias de tiempo
        X, y = crear_secuencias(group_data, n_steps, step_ahead)

        if len(X) == 0 or len(y) == 0:
            continue

        # Escalar los datos de entrada
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)

        # Construir y entrenar el modelo LSTM
        model = build_lstm_model((X_scaled.shape[1], X_scaled.shape[2]))
        model.fit(X_scaled, y, epochs=100, verbose=0)

        # Guardar el modelo
        model_key = (cat1, cat2, cat3, brand)
        models[model_key] = model

        # Predecir las ventas para los próximos 2 meses (ejemplo)
        X_pred = group_data[['tn']].values[-(n_steps+step_ahead):-step_ahead].reshape((1, n_steps, 1))
        X_pred_scaled = scaler.transform(X_pred.reshape(-1, 1)).reshape(X_pred.shape)
        pred = model.predict(X_pred_scaled, verbose=0)

        # Desescalar la predicción usando el scaler original para 'tn'
        pred_original_scale = scalers['tn'].inverse_transform(pred.reshape(-1, 1)).flatten()
        
        # Asegurarse de que las predicciones no sean negativas
        pred_original_scale = np.clip(pred_original_scale, 0, None)
        
        predictions.append([cat1, cat2, cat3, brand, pred_original_scale[0]])

    return models, predictions

# Llamada a la función con el DataFrame scaled_df
n_steps = 13
step_ahead = 2
models, predictions = process_data_in_batches(scaled_df, n_steps, step_ahead)

# Convertir las predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediction'])

# Sumarizar las predicciones por 'cat1', 'cat2', 'cat3', 'brand'
summarized_preds = predictions_df.groupby(['cat1', 'cat2', 'cat3', 'brand'])['prediction'].sum().reset_index()

# Paso 9: Aplicar los ratios para obtener las predicciones finales por product_id
final_predictions = []
for _, row in summarized_preds.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'])
    for (cat1, cat2, cat3, brand, product_id), ratio in ratio_dict.items():
        if (cat1, cat2, cat3, brand) == key:
            final_predictions.append([product_id, row['prediction'] * ratio])

# Convertir las predicciones finales a un DataFrame
final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediction'])

# Calcular el promedio de tn para los últimos 12 meses para cada product_id
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')
last_12_months = df[df['periodo'] >= df['periodo'].max() - pd.DateOffset(months=12)]
average_tn_last_12_months = last_12_months.groupby('product_id')['tn'].mean().reset_index()

# Identificar los product_id faltantes en final_predictions_df
missing_product_ids = set(df['product_id']) - set(final_predictions_df['product_id'])

# Agregar los product_id faltantes con el promedio de tn de los últimos 12 meses
for product_id in missing_product_ids:
    avg_tn = average_tn_last_12_months.loc[average_tn_last_12_months['product_id'] == product_id, 'tn']
    if not avg_tn.empty:
        final_predictions_df = pd.concat([final_predictions_df, pd.DataFrame({'product_id': [product_id], 'prediction': [avg_tn.values[0]]})], ignore_index=True)

# Guardar las predicciones finales en un archivo CSV
final_predictions_df.to_csv("C:/Users/Usuario/desktop/vero2/predicciones_LSTM_GRUPOS_7v8.csv", index=False)

print(final_predictions_df.head())